#CS430 - Individual Project 1
##**US Cars Dataset**
Gracie Bayer - October 20, 2022

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv('/content/USA_cars_datasets.csv')

In [3]:
data.head()

,Unnamed: 0,price,brand,model,year,title_status,mileage,color,vin,lot,state,country,condition
0,0,6300,toyota,cruiser,2008,clean vehicle,274117.0,black,jtezu11f88k007763,159348797,new jersey,usa,10 days left
1,1,2899,ford,se,2011,clean vehicle,190552.0,silver,2fmdk3gc4bbb02217,166951262,tennessee,usa,6 days left
2,2,5350,dodge,mpv,2018,clean vehicle,39590.0,silver,3c4pdcgg5jt346413,167655728,georgia,usa,2 days left
3,3,25000,ford,door,2014,clean vehicle,64146.0,blue,1ftfw1et4efc23745,167753855,virginia,usa,22 hours left
4,4,27700,chevrolet,1500,2018,clean vehicle,6654.0,red,3gcpcrec2jg473991,167763266,florida,usa,22 hours left


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2499 entries, 0 to 2498
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    2499 non-null   int64  
 1   price         2499 non-null   int64  
 2   brand         2499 non-null   object 
 3   model         2499 non-null   object 
 4   year          2499 non-null   int64  
 5   title_status  2499 non-null   object 
 6   mileage       2499 non-null   float64
 7   color         2499 non-null   object 
 8   vin           2499 non-null   object 
 9   lot           2499 non-null   int64  
 10  state         2499 non-null   object 
 11  country       2499 non-null   object 
 12  condition     2499 non-null   object 
dtypes: float64(1), int64(4), object(8)
memory usage: 253.9+ KB


In [5]:
data.describe()

,Unnamed: 0,price,year,mileage,lot
count,2499.000000,2499.000000,2499.000000,2.499000e+03,2.499000e+03
mean,1249.000000,18767.671469,2016.714286,5.229869e+04,1.676914e+08
std,721.543484,12116.094936,3.442656,5.970552e+04,2.038772e+05
min,0.000000,0.000000,1973.000000,0.000000e+00,1.593488e+08
25%,624.500000,10200.000000,2016.000000,2.146650e+04,1.676253e+08
50%,1249.000000,16900.000000,2018.000000,3.536500e+04,1.677451e+08
75%,1873.500000,25555.500000,2019.000000,6.347250e+04,1.677798e+08
max,2498.000000,84900.000000,2020.000000,1.017936e+06,1.678055e+08


In [6]:
data=data.drop(data[['Unnamed: 0','vin','lot','state','country','condition']],axis=1)

In [7]:
data=pd.get_dummies(data[['price','brand','model','year','title_status','mileage','color']],drop_first=True)

In [8]:
data.isnull().sum()

price                                     0
year                                      0
mileage                                   0
brand_audi                                0
brand_bmw                                 0
                                         ..
color_turquoise                           0
color_tuxedo black metallic               0
color_white                               0
color_white platinum tri-coat metallic    0
color_yellow                              0
Length: 205, dtype: int64

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2499 entries, 0 to 2498
Columns: 205 entries, price to color_yellow
dtypes: float64(1), int64(2), uint8(202)
memory usage: 551.7 KB


In [10]:
X=data.drop('price', axis=1)
y=data['price']

###Train Test Split & Scaling the Data

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=.20, random_state=42)

In [12]:
from sklearn.preprocessing import StandardScaler
sc_X=StandardScaler()
sc_y=StandardScaler()

###Importing libraries

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [14]:
from sklearn.ensemble import VotingClassifier

##Voting Classifier - ***Soft***

In [18]:
log_clf=LogisticRegression(solver='liblinear',random_state=42)
svm_clf=SVC(gamma='auto',probability=True)
dt_clf = DecisionTreeClassifier(random_state=0)
rnd_clf=RandomForestClassifier(n_estimators=5, random_state=42)

In [19]:
voting_clf=VotingClassifier(
    estimators=[('lr',log_clf),
                ('svc',svm_clf),
                ('dt',dt_clf),
                ('rf',rnd_clf)],
    voting='soft')
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(random_state=42,
                                                 solver='liblinear')),
                             ('svc', SVC(gamma='auto', probability=True)),
                             ('dt', DecisionTreeClassifier(random_state=0)),
                             ('rf',
                              RandomForestClassifier(n_estimators=5,
                                                     random_state=42))],
                 voting='soft')

In [22]:
from sklearn.metrics import accuracy_score
for clf in(log_clf,svm_clf,dt_clf,rnd_clf,voting_clf):
  clf.fit(X_train,y_train)
  y_pred=clf.predict(X_test)
  print(clf.__class__.__name__,accuracy_score(y_test,y_pred))

LogisticRegression 0.026
SVC 0.022
DecisionTreeClassifier 0.052
RandomForestClassifier 0.044
VotingClassifier 0.054


##Voting Classifier - ***Hard***

In [26]:
log_clf=LogisticRegression(solver='liblinear',random_state=42)
svm_clf=SVC(gamma='auto')
dt_clf = DecisionTreeClassifier(random_state=0)
rnd_clf=RandomForestClassifier(n_estimators=5, random_state=42)

In [27]:
voting_clf=VotingClassifier(
    estimators=[('lr',log_clf),
                ('svc',svm_clf),
                ('dt',dt_clf),
                ('rf',rnd_clf)],
    voting='hard')
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(random_state=42,
                                                 solver='liblinear')),
                             ('svc', SVC(gamma='auto')),
                             ('dt', DecisionTreeClassifier(random_state=0)),
                             ('rf',
                              RandomForestClassifier(n_estimators=5,
                                                     random_state=42))])

In [28]:
from sklearn.metrics import accuracy_score
for clf in(log_clf,svm_clf,dt_clf,rnd_clf,voting_clf):
  clf.fit(X_train,y_train)
  y_pred=clf.predict(X_test)
  print(clf.__class__.__name__,accuracy_score(y_test,y_pred))

LogisticRegression 0.026
SVC 0.022
DecisionTreeClassifier 0.052
RandomForestClassifier 0.044
VotingClassifier 0.044
